# 零基础实战机器学习

## 第17讲 集成学习-裂变转化预测

作者 黄佳

极客时间专栏链接：https://time.geekbang.org/column/intro/438


问题：通过集成学习方法预测用户是否会购买促销品

易速鲜花公司推出两张促销方案，分别是疯狂打折和买一送一。

通过集成学习方法，我们可以预测出用户是否会购买这些打折商品。


In [ ]:
import pandas as pd #导入Pandas
import numpy as np #导入NumPy
df_fission = pd.read_csv('易速鲜花裂变转化.csv') #载入数据
print('用户数:', df_fission.count()['用户码']) #查看数据条目数
df_fission #显示头几行数据

In [ ]:
# 把二元类别文本数字化
df_fission['性别'].replace("女",0,inplace = True)
df_fission['性别'].replace("男",1,inplace=True)
# 显示数字类别
print("Gender unique values",df_fission['性别'].unique())
# 把多元类别转换成多个二元哑变量，然后贴回原始数据集
df_fission = pd.get_dummies(df_fission, drop_first = True)
# df_fission = [df_fission, d_city]
# df_fission = pd.concat(df_bank, axis = 1)
# 构建特征和标签集合
# y = df_fission['Exited']
# X = df_fission.drop(['Name', 'Exited', 'City'], axis=1)
# X.head() #显示新的特征集
df_fission

In [ ]:
import matplotlib.pyplot as plt #导入pyplot模块
import seaborn as sns #导入Seaborn
fig = sns.countplot(x='是否转化', data=df_fission) #创建柱状计数图
# fig.set_xticklabels(fig.get_xticklabels(),rotation=25) #X轴标签倾斜
fig.set_ylabel("数目") #Y轴标题
plt.show() #显示图像

### 构建特征和标签数据集

In [ ]:
# df_fission = df_fission.query("裂变类型 == '助力砍价'")

In [ ]:
X = df_fission.drop(['用户码','是否转化'], axis = 1) # 构建特征集
y = df_fission.是否转化.values # 构建标签集

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.preprocessing import MinMaxScaler #导入归一化缩放器
scaler = MinMaxScaler() #创建归一化缩放器
X_train = scaler.fit_transform(X_train) #拟合并转换训练集数据
X_test = scaler.transform(X_test) #转换测试集数据

In [ ]:
X_train

In [ ]:
from sklearn.metrics import confusion_matrix # 导入混淆矩阵
import seaborn as sns #导入seaborn画图工具箱
def show_matrix(y_test, y_pred): # 定义一个函数显示混淆矩阵
    cm = confusion_matrix(y_test,y_pred) # 调用混淆矩阵
    plt.title("混淆矩阵") # 标题
    sns.heatmap(cm,annot=True,cmap="Blues",fmt="d",cbar=False) # 热力图设定
    plt.show() # 显示混淆矩阵

In [ ]:
# 对多棵决策树进行聚合(Bagging)
from sklearn.ensemble import BaggingClassifier # 导入Bagging 分类器
from sklearn.tree import DecisionTreeClassifier # 导入决策树分类器
from sklearn.metrics import (f1_score, confusion_matrix) # 导入评估指标
dt = BaggingClassifier(DecisionTreeClassifier()) # 只使用一棵决策树
dt.fit(X_train, y_train) # 拟合模型
y_pred = dt.predict(X_test) # 进行预测
print(" 决策树测试准确率: {:.2f}%".format(dt.score(X_test, y_test)*100))
print(" 决策树测试F1 分数: {:.2f}%".format(f1_score(y_test, y_pred)*100))
bdt = BaggingClassifier(DecisionTreeClassifier()) # 树的Bagging
bdt.fit(X_train, y_train) # 拟合模型
y_pred = bdt.predict(X_test) # 进行预测
print(" 决策树Bagging 测试准确率: {:.2f}%".format(bdt.score(X_test, y_test)*100))
print(" 决策树Bagging 测试F1 分数: {:.2f}%".format(f1_score(y_test, y_pred)*100))
show_matrix(y_test, y_pred)
f1_bdt = f1_score(y_test, y_pred)*100

In [ ]:
from sklearn.model_selection import GridSearchCV # 导入网格搜索工具
from sklearn.ensemble import RandomForestClassifier # 导入随机森林模型
rf = RandomForestClassifier() # 随机森林模型
rf.fit(X_train, y_train) # 拟合模型
y_pred = rf.predict(X_test) # 进行预测
print(" 随机森林测试准确率: {:.2f}%".format(rf.score(X_test, y_test)*100))
print(" 随机森林测试F1 分数: {:.2f}%".format(f1_score(y_test, y_pred)*100))
show_matrix(y_test, y_pred)
f1_rf = f1_score(y_test, y_pred)*100

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier # 导入极端随机森林模型
ext = ExtraTreesClassifier() # 极端随机森林模型
ext.fit(X_train, y_train) # 拟合模型
y_pred = ext.predict(X_test) # 进行预测
print(" 极端随机森林测试准确率: {:.2f}%".format(ext.score(X_test, y_test)*100))
print(" 极端随机森林测试F1 分数: {:.2f}%".format(f1_score(y_test, y_pred)*100))
show_matrix(y_test, y_pred)
f1_ext = f1_score(y_test, y_pred)*100

In [ ]:
from sklearn.ensemble import AdaBoostClassifier # 导入AdaBoost 模型
dt = DecisionTreeClassifier() # 选择决策树分类器作为AdaBoost 的基准算法
ada = AdaBoostClassifier(dt) # AdaBoost 模型
ada.fit(X_train, y_train) # 拟合模型
y_pred = ada.predict(X_test) # 进行预测
print("AdaBoost 测试准确率: {:.2f}%".format(ada.score(X_test, y_test)*100))
print("AdaBoost 测试F1 分数: {:.2f}%".format(f1_score(y_test, y_pred)*100))
show_matrix(y_test, y_pred)
f1_ada = f1_score(y_test, y_pred)*100

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier # 导入梯度提升模型
gb = GradientBoostingClassifier() # 梯度提升模型
gb.fit(X_train, y_train) # 拟合模型
y_pred = gb.predict(X_test) # 进行预测
print(" 梯度提升测试准确率: {:.2f}%".format(gb.score(X_test, y_test)*100))
print(" 梯度提升测试F1 分数: {:.2f}%".format(f1_score(y_test, y_pred)*100))
show_matrix(y_test, y_pred)
f1_gb = f1_score(y_test, y_pred)*100

In [ ]:
from xgboost import XGBClassifier # 导入XGB 模型
xgb = XGBClassifier() # XGB 模型
xgb.fit(X_train, y_train) # 拟合模型
y_pred = xgb.predict(X_test) # 进行预测
print("XGB 测试准确率: {:.2f}%".format(xgb.score(X_test, y_test)*100))
print("XGB 测试F1 分数: {:.2f}%".format(f1_score(y_test, y_pred)*100))
show_matrix(y_test, y_pred)
f1_xgb = f1_score(y_test, y_pred)*100

In [ ]:
# from xgboost import XGBClassifier # 导入XGB 模型
# xgb = XGBClassifier() # XGB 模型
# # 使用网格搜索优化参数
# xgb_param_grid = {'min_child_weight': [1, 5, 10],
# 'gamma': [0.5, 1, 1.5, 2, 5],
# 'subsample': [0.6, 0.8, 1.0],
# 'colsample_bytree': [0.6, 0.8, 1.0],
# 'max_depth': [3, 4, 5]}
# xgb_gs = GridSearchCV(xgb, param_grid = xgb_param_grid,
# scoring="f1", n_jobs= 10, verbose = 1)
# xgb_gs.fit(X_train, y_train) # 拟合模型
# xgb_gs = xgb_gs.best_estimator_ # 最佳模型
# y_pred = xgb_gs.predict(X_test) # 进行预测
# print("XGB 测试准确率: {:.2f}%".format(xgb_gs.score(X_test, y_test)*100))
# print("XGB 测试F1分数: {:.2f}%".format(f1_score(y_test, y_pred)*100))
# show_matrix(y_test, y_pred)

In [ ]:
from sklearn.ensemble import VotingClassifier # 导入Voting 模型
# 把各种模型的预测结果进行Voting。同学们还可以加入更多模型如SVM, KNN 等
voting = VotingClassifier(estimators=[('rf', rf),('bdt', bdt),('gb', gb),
                                      ('ext', ext),('xgb', xgb),('ada', ada)],
voting='soft', n_jobs=10)
voting = voting.fit(X_train, y_train) # 拟合模型
y_pred = voting.predict(X_test) # 进行预测
print("Voting 测试准确率: {:.2f}%", voting.score(X_test, y_test)*100)
print("Voting 测试F1分数: {:.2f}%", f1_score(y_test, y_pred)*100)

In [ ]:
# from sklearn.model_selection import GridSearchCV # 导入网格搜索工具
# from sklearn.ensemble import RandomForestClassifier # 导入随机森林模型
# rf = RandomForestClassifier() # 随机森林模型
# # 使用网格搜索优化参数
# rf_param_grid = {"max_depth": [None],
# "max_features": [1, 3, 10],
# "min_samples_split": [2, 3, 10],
# "min_samples_leaf": [1, 3, 10],
# "bootstrap": [True, False],
# "n_estimators" :[100, 300],
# "criterion": ["gini"]}
# rf_gs = GridSearchCV(rf, param_grid = rf_param_grid,
# scoring="f1", n_jobs= 10, verbose = 1)
# rf_gs.fit(X_train, y_train) # 拟合模型
# rf_gs = rf_gs.best_estimator_ # 最佳模型
# y_pred = rf_gs.predict(X_test) # 进行预测
# print("调优后随机森林测试准确率: {:.2f}%".format(rf_gs.score(X_test, y_test)*100))
# print("调优后随机森林测试F1分数: {:.2f}%".format(f1_score(y_test, y_pred)*100))

In [ ]:
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

In [ ]:
import keras # 导入Keras库
from keras.models import Sequential # 导入Keras序贯模型
from keras.layers import Dense # 导入Keras密集连接层
ann = Sequential() # 创建一个序贯ANN模型
ann.add(Dense(units=12, input_dim=11, activation = 'relu')) # 添加输入层
ann.add(Dense(units=24, activation = 'relu')) # 添加隐层
ann.add(Dense(units=48, activation = 'relu')) # 添加隐层
ann.add(Dense(units=1, activation = 'sigmoid')) # 添加输出层
ann.summary() # 显示网络模型（这个语句不是必须的）
# 编译神经网络，指定优化器，损失函数，以及评估标准
ann.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['acc'])
history = ann.fit(X_train, y_train, # 指定训练集
                  epochs=30,        # 指定训练的轮次
                  batch_size=64,    # 指定数据批量
                  validation_split=0.2) #指定验证集,这里为了简化模型，直接用训练集数据
# show_history(history) # 调用这个函数

In [ ]:
y_pred = ann.predict(X_test,batch_size=10) # 预测测试集的标签
y_pred = np.round(y_pred) # 将分类概率值转换成0/1整数值
print("神经网络测试F1分数: {:.2f}%".format(f1_score(y_test, y_pred)*100))

In [ ]:
# plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
# plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
# plt.rcParams["font.family"]=['SimHei']
methods = ["树的Bagging", "随机森林", "极端随机森林",
"AdaBoost", "GBDT", "XGBoost"]
f1 = [f1_bdt, f1_rf, f1_ext, f1_ada, f1_gb, f1_xgb]
colors = ["orange", "red", "purple", "magenta", "green", "blue"]
# sns.set_style("whitegrid")
plt.figure(figsize=(16, 5))
plt.yticks(np.arange(0, 100, 10))
plt.ylabel("F1分数")
plt.xlabel("算法")
sns.barplot(x=methods, y=f1, palette=colors)
# plt.grid(b=None)
plt.show()